In [ ]:
!pip install -q tf-models-official==2.4.0
import io
import os
import re
import shutil
import string
import tensorflow as tf
from tqdm import tqdm

#use for naive_bayes
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer,SnowballStemmer
from nltk.tokenize import TweetTokenizer
import pdb
from nltk.corpus import twitter_samples
import numpy as np
import pandas as pd
import nltk
from os import getcwd

from tensorflow.keras import layers
from tensorflow.keras import losses

nltk.download('stopwords')

#use for LSTM and WordEmbedding Model
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input, Dropout, LSTM, Activation
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.initializers import glorot_uniform

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer


import os

import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf

import tensorflow_hub as hub
import tensorflow_datasets as tfds
tfds.disable_progress_bar()

from official.modeling import tf_utils
from official import nlp
from official.nlp import bert

# Load the required submodules
import official.nlp.optimization
import official.nlp.bert.bert_models
import official.nlp.bert.configs
import official.nlp.bert.run_classifier
import official.nlp.bert.tokenization
import official.nlp.data.classifier_data_lib
import official.nlp.modeling.losses
import official.nlp.modeling.models
import official.nlp.modeling.networks

     |████████████████████████████████| 1.1 MB 4.2 MB/s 
     |████████████████████████████████| 234 kB 56.2 MB/s 
     |████████████████████████████████| 1.2 MB 47.6 MB/s 
     |████████████████████████████████| 99 kB 8.2 MB/s 
     |████████████████████████████████| 352 kB 51.0 MB/s 
     |████████████████████████████████| 596 kB 56.3 MB/s 
     |████████████████████████████████| 1.1 MB 34.7 MB/s 
     |████████████████████████████████| 43 kB 1.9 MB/s 
     |████████████████████████████████| 47.8 MB 2.2 MB/s 
     |████████████████████████████████| 462 kB 58.6 MB/s 
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
train_data_path = "/content/drive/MyDrive/Big_Data/custom_model/test_data.csv"
test_data_path = "/content/drive/MyDrive/Big_Data/custom_model/test_data.csv"

In [ ]:
train_data = pd.read_csv(train_data_path,delimiter=',')
test_data = pd.read_csv(test_data_path,delimiter = ',') 

In [ ]:
train_data.head()

,Unnamed: 0,Unnamed: 0.1,target,ids,date,user,text
0,1435527,1435527,POSITIVE,2060822046,Sat Jun 06 19:38:40 PDT 2009,GiGGLESx912,aha problem
1,1444705,1444705,POSITIVE,2062290864,Sat Jun 06 22:49:32 PDT 2009,marufumega,NaN
2,678206,678206,NEGATIVE,2248906461,Fri Jun 19 21:23:11 PDT 2009,Waylin,time find sudaf help sinus pressur relat tooth...
3,845944,845944,POSITIVE,1564314457,Mon Apr 20 02:52:51 PDT 2009,Hollywood_Trey,hahahahha classic lol u r waaay 2 sexxxi 2 2 e...
4,1166075,1166075,POSITIVE,1979899777,Sun May 31 04:57:15 PDT 2009,bella456,hey i'm still haha rt @5tosucce hey parti anim


In [ ]:
train_data["text"] = str(train_data["text"])
test_data["text"] = str(test_data["text"])

In [ ]:
#create vocab
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_data.text)
# vocab_size = len(tokenizer.word_index) + 1

#create data
x_train = pad_sequences(tokenizer.texts_to_sequences(train_data.text), maxlen= 300)
x_test = pad_sequences(tokenizer.texts_to_sequences(test_data.text), maxlen= 300)

#create y_data
encoder = LabelEncoder()
encoder.fit(train_data.target.tolist())

y_train = encoder.transform(train_data.target.tolist())
y_test = encoder.transform(test_data.target.tolist())

In [ ]:
gs_folder_bert = "gs://cloud-tpu-checkpoints/bert/v3/uncased_L-12_H-768_A-12"
tf.io.gfile.listdir(gs_folder_bert)

['bert_config.json',
 'bert_model.ckpt.data-00000-of-00001',
 'bert_model.ckpt.index',
 'vocab.txt']

In [ ]:
hub_url_bert = "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3"